<a href="https://colab.research.google.com/github/phpons/reversii-player/blob/master/Reversi_player.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import random
import sys
import copy

In [ ]:
starting_board = """........
........
........
...WB...
...BW...
........
........
........"""

In [ ]:
f = open('board.txt', 'w')
f.write(starting_board)
f.close()

# Predefined

In [ ]:
def from_file(path_to_file):
    """
    Generates a board from the string representation
    contained in the file
    :param path_to_file:
    :return: Board object
    """
    return from_string(open(path_to_file).read())


def from_string(string):
    """
    Generates a board from the string representation
    :param string:
    :return:
    """
    b = Board()
    # resets piece_count and set it during board construction
    b.piece_count = {b.BLACK: 0, b.WHITE: 0, b.EMPTY: 0}
    for lineno, line in enumerate(string.split('\n')):
        line.strip()  # cuts the \n

        for colno, col in enumerate(line):
            b.tiles[lineno][colno] = col
            b.piece_count[col] += 1

    return b

class Board(object):
    """
    Board implementation strongly inspired by: http://dhconnelly.com/paip-python/docs/paip/othello.html
    """

    BLACK = 'B'
    WHITE = 'W'
    EMPTY = '.'

    # direction of neighbor tiles (add to current tile coordinates to obtain neighbor)
    UP = (0, -1)
    DOWN = (0, 1)
    LEFT = (-1, 0)
    RIGHT = (1, 0)
    UP_LEFT = (-1, -1)
    UP_RIGHT = (1, -1)
    DOWN_LEFT = (-1, 1)
    DOWN_RIGHT = (1, 1)

    # list with all directions
    DIRECTIONS = [UP, DOWN, LEFT, RIGHT, UP_LEFT, UP_RIGHT, DOWN_LEFT, DOWN_RIGHT]

    def __init__(self):
        """
        Initializes the 8x8 board with all tiles empty, except the center
        that are initialized according to othello's initial board
        :return:
        """
        self.tiles = [[self.EMPTY] * 8 for i in range(8)]

        self.tiles[3][3], self.tiles[3][4] = self.WHITE, self.BLACK
        self.tiles[4][3], self.tiles[4][4] = self.BLACK, self.WHITE

        # cache legal moves in attempt to reduce function calls
        self._legal_moves = {self.BLACK: None, self.WHITE: None}

        self.piece_count = {self.BLACK: 2, self.WHITE: 2, self.EMPTY: 60}

    def is_within_bounds(self, move):
        """
        Returns whether the move refers to a valid board position
        :param move: (int, int)
        :return: bool
        """
        return 0 <= move[0] < 8 and 0 <= move[1] < 8

    def is_legal(self, move, color):
        """
        Returns whether the move is legal for the given color
        :param move: (int,int) tile position to place the disk
        :param color: color of the player making the move
        :return: bool
        """
        # move is queried row,col but stored col,row in legal_moves
        return (move[1], move[0]) in self.legal_moves(color)

    def find_bracket(self, move, color, direction):
        """
        Traverses the board in given direction trying to
        find a tile of the given color that surrounds opponent tiles, returns
        :param move: (int, int)
        :param color: color of player making the move
        :param direction: one of eight directions of tile neighborhood
        :return: (int,int)
        """
        # performing inline boundary check to avoid calls for is_within_bounds
        # this saves some time
        dx, dy = direction
        tx, ty = move
        tx += dx
        ty += dy

        opp = self.BLACK if color == self.WHITE else self.WHITE  # inline opponent calc.

        if not (0 <= tx <= 7 and 0 <= ty <= 7) or self.tiles[tx][ty] != opp:
            return False

        while self.tiles[tx][ty] == opp:  # putting is_within_bounds here yields more calls
            tx += dx
            ty += dy
            if not (0 <= tx <= 7 and 0 <= ty <= 7):  # self.is_within_bounds((tx, ty)):
                return False

        if self.tiles[tx][ty] == self.EMPTY:
            return False
        return tx, ty

    def find_where_to_play_from_owned(self, owned, color, direction):
        """
        Traverses the board in given direction trying to
        find an empty tile that surrounds opponent tiles, returns it.
        This is the dual of find_bracket, which goes from empty to
        the piece of the desired color
        :param owned: (int, int), col, row coordination of owned tile
        :param color: color of owned tile
        :param direction: one of eight directions of tile neighborhood
        :return: (int,int) or False if not found
        """
        # performing inline boundary check to avoid calls for is_within_bounds
        # this saves some time
        dx, dy = direction
        tx, ty = owned
        tx += dx
        ty += dy
        opp = self.BLACK if color == self.WHITE else self.WHITE  # inline opponent calc.

        if not (0 <= tx <= 7 and 0 <= ty <= 7) or self.tiles[tx][ty] != opp:  # color:
            return False

        while self.tiles[tx][ty] == opp:
            tx += dx
            ty += dy
            if not (0 <= tx <= 7 and 0 <= ty <= 7):
                return False

        if self.tiles[tx][ty] != self.EMPTY:
            return False
        return tx, ty

    def process_move(self, position, color):
        """
        Executes the placement of a tile of a given color
        in a given position
        :param position:
        :param color:
        :return: bool
        """

        # as the board is represented row-column, swaps coords to col-row
        position = position[1], position[0]

        if color not in [self.WHITE, self.BLACK]:
            raise ValueError("Move must be made by BLACK or WHITE player")

        if self.is_legal(position, color):
            # places the piece and update piece counts
            px, py = position
            self.tiles[px][py] = color
            self.piece_count[color] += 1
            self.piece_count[self.EMPTY] -= 1

            for direc in self.DIRECTIONS:
                self.flip_tiles(position, color, direc)

            # resets legal moves
            self._legal_moves[self.BLACK], self._legal_moves[self.WHITE] = None, None
            return True

        return False  # guards against illegal moves

    def flip_tiles(self, origin, color, direction):
        """
        Traverses the board in the given direction,
        transforming the color of appropriate tiles
        :param origin: where the traversal will begin
        :param color:
        :param direction:
        :return:
        """
        destination = self.find_bracket(origin, color, direction)  # move, player, board, direction)
        if not destination:
            return

        ox, oy = origin
        dx, dy = direction

        nx, ny = ox + dx, oy + dy  # n stands for 'next'

        opp = self.opponent(color)

        while (nx, ny) != destination:
            # flips the tile and updates piece counts
            self.tiles[nx][ny] = color
            self.piece_count[color] += 1
            self.piece_count[opp] -= 1
            nx, ny = nx + dx, ny + dy

    def legal_moves(self, color):
        """
        Returns a list of legal moves for the given color
        :param color:
        :return:
        """
        if self._legal_moves[color] is None:
            # construct the list of legal moves only once
            self._legal_moves[color] = []

            if self.piece_count[color] > self.piece_count[self.EMPTY]:
                self.find_legal_moves_dense(color)
            else:
                self.find_legal_moves_sparse(color)

        return self._legal_moves[color]

    def find_legal_moves_dense(self, color):
        """
        Finds the legal moves for a given color in a dense board.
        A dense board has less empty tiles than pieces of the given color.
        :param color:
        """
        # test if every empty tile on the board is a legal move
        tiles = [(x, y) for x in range(8) for y in range(8) if self.tiles[x][y] == self.EMPTY]

        for x, y in tiles:
            if self.tiles[x][y] == self.EMPTY:  # and any(map(hasbracket, self.DIRECTIONS)):
                # performs the 'inline' any:
                for direc in self.DIRECTIONS:
                    if self.find_bracket((x, y), color, direc):
                        # flips x,y because of the way tiles are stored and the x,y coords in real world
                        self._legal_moves[color].append((y, x))
                        break

    def find_legal_moves_sparse(self, color):
        """
        Finds the legal moves for a given color in a sparse board.
        A sparse board has more empty tiles than pieces of the given color
        :param color:
        :return:
        """
        # test if every empty tile on the board is a legal move
        tiles = [(x, y) for x in range(8) for y in range(8) if self.tiles[x][y] == color]

        for x, y in tiles:
            if self.tiles[x][y] == color:
                for direc in self.DIRECTIONS:
                    move = self.find_where_to_play_from_owned((x, y), color, direc)
                    if move:
                        # flips x,y because of the way tiles are stored and the x,y coords in real world
                        self._legal_moves[color].append((move[1], move[0]))

    def has_legal_move(self, color):
        """
        Returns whether the given color has any legal move
        :param color:
        :return:bool
        """
        # test if every empty tile on the board is a legal move
        tiles = [(x, y) for x in range(8) for y in range(8) if self.tiles[x][y] == self.EMPTY]

        for x, y in tiles:
            # self._legal_moves[color] = [(y, x) for x, y in tiles if self.is_legal((x, y), color)]

            hasbracket = lambda direction: self.find_bracket((x, y), color, direction)

            if self.tiles[x][y] == self.EMPTY and any(map(hasbracket, self.DIRECTIONS)):
                return True
        return False

    def opponent(self, color):
        """
        Returns the opponent of the received color
        :param color:
        :return:
        """
        if color == self.EMPTY:
            raise ValueError('Empty has no opponent.')

        if color == self.WHITE:
            return self.BLACK
        else:
            return self.WHITE

    def print_board(self):
        """
        Prints the string representation of the board
        :return:
        """

        print(self.decorated_str())

    def decorated_str(self):
        """
        Returns the string representation of the board
        decorated with coordinates for board positions
        :return: str
        """
        string = 'x 01234567\n'
        for i, row in enumerate(self.tiles):
            string += '%d %s\n' % (i, ''.join(row))

        return string

    def __str__(self):
        """
        Returns the string representation of the board
        :return: str
        """
        string = ''
        for i, row in enumerate(self.tiles):
            string += '%s\n' % ''.join(row)

        return string


In [ ]:
import random
import sys
sys.path.append('..')

class RandomPlayer:
  def __init__(self, the_board, color):
    self.board_path = board_path

    self.color = Board.WHITE if color == 'white' else Board.BLACK
    self.the_board = the_board


  def make_move(self):
      """
      Returns a random move from the list of possible ones
      :return: (int, int)
      """
      legal_moves = self.the_board.legal_moves(self.color)

      return random.choice(legal_moves) if len(legal_moves) > 0 else (-1, -1)
      

  def play(self):
      f = open('move.txt', 'w')
      move = self.make_move()
      f.write('%d,%d' % move)
      f.close()
      return move
    


In [ ]:
class BoardNode:
	def __init__(self, y, x, parent = None):
		self.y = y
		self.x = x
		self.parent = parent
		self.value = 0
		self.children = []

In [ ]:
class MyPlayer:
  def __init__(self, the_board, color):
    self.board_path = board_path

    self.color = Board.WHITE if color == 'white' else Board.BLACK
    self.the_board = the_board
    self.maxDepth = 3
    self.board_values_map = [
      [160, 20, 60, 45, 45, 60, 20, 160],
      [20, 0, 35, 35, 35, 35, 0, 20],
      [60, 35, 55, 43, 43, 55, 35, 60],
      [45, 35, 43, 43, 43, 43, 35, 45],
      [45, 35, 43, 43, 43, 43, 35, 45],
      [60, 35, 55, 43, 43, 55, 35, 60],
      [20, 0, 35, 35, 35, 35, 0, 20],
      [160, 20, 60, 45, 45, 60, 20, 160]
    ]

  def utility(self, move, the_board, color):
    return 1 * self.position_value(move) ** 3 + self.possibleMovementsCount(the_board, color) ** 3

  def possibleMovementsCount(self, the_board, color):
    return len(the_board.legal_moves(color))
  
  def position_value(self, move):
    column, row = move
    return self.board_values_map[row][column]

  def make_move(self, the_board):
      """
      Returns a random move from the list of possible ones
      :return: (int, int)
      """
      # color = board.Board.WHITE if color == 'white' else board.Board.BLACK
      legal_moves = the_board.legal_moves(self.color)

      return self.minimax_decision(the_board, 0)

  def minimax_decision(self, the_board, depth):
      our_board = copy.deepcopy(the_board)
      finalValue, finalMove = self.max_value(our_board, depth, -999999999, 999999999, 0)
      print('MAX VALUE :) ', finalValue, "Best move:", finalMove)		
      return finalMove

  def max_value(self, the_board, depth, alpha, beta, pathValue):
    if depth > self.maxDepth or not the_board.has_legal_move(self.color) and depth != 0:
        return pathValue

    for possibleMove in the_board.legal_moves(self.color):
      newBoard = copy.deepcopy(the_board)
      newBoard.process_move(possibleMove, self.color)
      moveValue = self.min_value(newBoard, depth + 1, alpha, beta, pathValue + self.utility(possibleMove, newBoard, self.color))

      if alpha < moveValue:
        alpha = moveValue
        bestMove = possibleMove

      if beta < alpha and depth == 0:
        return alpha, bestMove
      elif beta < alpha:
        return alpha

    if depth == 0:   
        if the_board.has_legal_move(self.color):
          print(alpha, bestMove) 
          return alpha, bestMove
        else:
          return alpha, (-1, -1)

    else:
        return alpha

  def min_value(self, the_board, depth, alpha, beta, pathValue):
    currentColor = the_board.opponent(self.color)
    if depth > self.maxDepth or not the_board.has_legal_move(currentColor):
        return pathValue

    for possibleMove in the_board.legal_moves(currentColor):
      newBoard = copy.deepcopy(the_board)
      newBoard.process_move(possibleMove, currentColor)
      beta = min(beta, self.max_value(newBoard, depth + 1, alpha, beta, pathValue - self.utility(possibleMove, newBoard, currentColor)))

      if beta < alpha:
        return beta

    return beta

  def play(self):
    f = open('move.txt', 'w')
    move = self.make_move(self.the_board)
    f.write('%d,%d' % move)
    f.close()
    return move


  def coinsAmount(self, our_board, color):
    return our_board.piece_count[color]

## Execution

In [ ]:
# Read the initial board file
board_path = "/content/board.txt"

In [ ]:
b = from_file(board_path)

In [ ]:
r_white = MyPlayer(b, 'white')
r_black = MyPlayer(b, 'black')

In [ ]:
import time

start = time.time()

n = 0
# Agents playing against each other
while n < 50:
  start = time.time()
  n += 1
  b.print_board()
  move = r_black.play()
  print(move)
  b.process_move(move, r_black.color)
  end = time.time()
  print("preto demorou", end - start, "s na jogada", n)


  b.print_board()
  move = r_white.play()
  b.process_move(move, r_white.color)

hello
hello
x 01234567
0 ........
1 ........
2 ........
3 ...WB...
4 ...BW...
5 ........
6 ........
7 ........

-208 (2, 3)
MAX VALUE :)  -208 Best move: (2, 3)
(2, 3)
preto demorou 0.0798189640045166 s na jogada 1
x 01234567
0 ........
1 ........
2 ........
3 ..BBB...
4 ...BW...
5 ........
6 ........
7 ........

139 (2, 2)
hello
x 01234567
0 ........
1 ........
2 ..W.....
3 ..BWB...
4 ...BW...
5 ........
6 ........
7 ........

133 (3, 2)
MAX VALUE :)  133 Best move: (3, 2)
(3, 2)
preto demorou 0.1599884033203125 s na jogada 2
x 01234567
0 ........
1 ........
2 ..WB....
3 ..BBB...
4 ...BW...
5 ........
6 ........
7 ........

91 (4, 2)
hello
x 01234567
0 ........
1 ........
2 ..WWW...
3 ..BBW...
4 ...BW...
5 ........
6 ........
7 ........

280 (5, 2)
MAX VALUE :)  280 Best move: (5, 2)
(5, 2)
preto demorou 0.6489114761352539 s na jogada 3
x 01234567
0 ........
1 ........
2 ..WWWB..
3 ..BBB...
4 ...BW...
5 ........
6 ........
7 ........

115 (3, 5)
hello
x 01234567
0 ........
1 ........


KeyboardInterrupt: ignored